# 예제: CNN을 이용한 MNIST 숫자 분류

---



CNN을 이용한 MNIST 숫자분류 실습을 하겠습니다.

먼저 텐서플로를 임포트해주세요.

In [ ]:
# 텐서플로 라이브러리를 임포트 합니다.
import tensorflow as tf
import sys
print("Tensorflow version: ", tf.__version__)   #2.18.0 as of 2025.04.08
print("Keras version: ", tf.keras.__version__)  #3.8.0 as of 2025.04.08
print("Python version: ", sys.version)          #3.11.11 as of 2025.04.08

다음에는 MNIST 데이터를 케라스 내장 데이터셋에서 다운로드 합니다.

다운로드한 이미지는 딥러닝 학습에 적합한 구조로 전처리를 해줘야 하는데, 먼저 이미지를 float type으로 바꿔주고, 28*28 2차원 이미지를  784개 1차원 벡터로 flattenin합니다.
그 후에, [0, 255]값을 갖는 데이터를 [0.0, 1.0]값으로 normalize해주면 이미지 데이터 x의 전처리 과정이 마무리됩니다. 레이블 데이터 y값은 one-hot-encoding을 적용해줍니다.


In [ ]:
# MNIST 데이터를 다운로드 합니다.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# 이미지들을 float32 데이터 타입으로 변경합니다.
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
# 28*28 형태의 이미지를 784차원으로 1*784로 flattening 합니다.
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])
# [0, 255] 사이의 값을 [0, 1]사이의 값으로 Normalize합니다.
x_train, x_test = x_train / 255., x_test / 255.
# 레이블 데이터에 one-hot encoding을 적용합니다.
y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)


다음으로는 6만개 데이터를 한꺼번에 메모리에 읽어서 처리하기에는 메모리 오버플로우 위험이 있기 때문에, 미니 배치로 쪼개서 트레이닝을 진행하기 위해 6만개를 무작위로 섞어서 50개로 쪼개는 작업을 진행합니다.


In [ ]:
# tf.data API를 이용해서 데이터를 섞고 batch 형태로 가져옵니다.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(60000).batch(50)
train_data_iter = iter(train_data)


이제 tf.keras.model API를 이용해서 매우 간단한 CNN모델의 class를 정의해줍니다.

이 CNN구조는 2개의 conv 레이어와 2개의 fully connected layer를 사용했는데, 맨처음 논문으로 발표된 CNN구조인 Alexnet이 5개의 conv레이어와 3개의 fully connected layer를 사용했고 그 후에 더 발전된 CNN구조들은 수십에서 수백개의 레이어를 사용했으므로, 여기 예제코드에서 정의된 구조는 매우 간소화된 CNN모델이라고 생각하시면 됩니다.

첫번째 컨브 레이어는 32개의 5x5 filter를 사용하고, stride=1, padding 은 입력 이미지와 같은 사이즈가 되도록 채우게 하고, activation function은 ReLU로 하겠다고 선언했습니다.
그 다음에는 맥스 풀링을 정의했는데, 2x2 filter size로 stride=2로 정의했으니까 출력 피처맵 사이즈는 입력보다 절반 사이즈가 될 것입니다.

두번째 컨브 레이어와 풀링 레이어 정의에서는 64개의 5x5 filter로 컨볼루션을 하고, 2x2 filter로 맥스 풀링을 하는데, 패딩이나 activation함수는 이전 레이어와 동일하게 사용합니다.

그 후에는 케라스의 flattening함수를 사용해서 flattening을 진행해줍니다. Flattening은 맨처음 CNN의 구조를 설명할 때에 잠깐 이야기했었는데, N개의 2차원 피처맵들을 1차원으로 쭉 펴주는 작업으로 이해하면 됩니다.

케라스 라이브러리를 이용해서 레이어들의 정의가 끝나면, 아래쪽에서 데이터 플로우를 정의하는데,
  

1.   28x28입력 이미지에 32개 필터를 사용하기 위해서 1x28x28x1로 reshape해주고,
2.   첫번째 컨브를 거치면 32개의 28x28 피처맵이 나오고,
3. 맥스 풀링을 거치면 피처맵이 절반인 14x14로 줄어들고,
4. 두번째 레이어의 컨브를 거치면 64개의 14x14 피처맵이 나오고,
5. 맥스 풀링을 거치면 피처맵이 절반 사이즈로 줄어서 64개의 7x7 피처맵들이 됩니다.
6. 이 피처맵들을 flattening을 하게 되면, 7x7x64, 즉 3136개의 노드로 바뀌게 됩니다.
7. 3136개의 피처들은 fully connected layer-1에서 1024개로 매핑이 되고,
8. 1024개는 다시 10개의 숫자 이미지를 의미하는 10개의 로짓으로 매핑이 됩니다.
9. 로짓값에 소프트맥스 함수를 적용하면 숫자 예측값이 나오게 됩니다.

이 CNN 클래스의 return값으로는 소프트맥스를 적용한 y_pred 예측값뿐 아니라, 소프트맥스를 적용하기 전의 로지트값도 같이 리턴하도록 정의했는데, 이유는 조금 있다가 설명하겠습니다.


In [ ]:
# tf.keras.Model을 이용해서 CNN 모델을 정의합니다.
class CNN(tf.keras.Model):
  def __init__(self):
    super(CNN, self).__init__()
    # 첫번째 Convolution Layer
    # 5x5 Kernel Size를 가진 32개의 Filter를 적용합니다.
    self.conv_layer_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=5, strides=1, padding='same', activation='relu')
    self.pool_layer_1 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)

    # 두번째 Convolutional Layer
    # 5x5 Kernel Size를 가진 64개의 Filter를 적용합니다.
    self.conv_layer_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=5, strides=1, padding='same', activation='relu')
    self.pool_layer_2 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)

    # Fully Connected Layer
    # 7x7 크기를 가진 64개의 activation map을 1024개의 특징들로 변환합니다.
    self.flatten_layer = tf.keras.layers.Flatten()
    self.fc_layer_1 = tf.keras.layers.Dense(1024, activation='relu')

    # Output Layer
    # 1024개의 특징들(feature)을 10개의 클래스-one-hot encoding으로 표현된 숫자 0~9-로 변환합니다.
    self.output_layer = tf.keras.layers.Dense(10, activation=None)

  def call(self, x):
    # MNIST 데이터를 3차원 형태로 reshape합니다. MNIST 데이터는 grayscale 이미지기 때문에 3번째차원(컬러채널)의 값은 1입니다.
    x_image = tf.reshape(x, [-1, 28, 28, 1])    # 28x28x1 -> 1x28x28x1
    h_conv1 = self.conv_layer_1(x_image)        # 1x28x28x1 -> 28x28x32
    h_pool1 = self.pool_layer_1(h_conv1)        # 28x28x32 -> 14x14x32
    h_conv2 = self.conv_layer_2(h_pool1)        # 14x14x32 -> 14x14x64
    h_pool2 = self.pool_layer_2(h_conv2)        # 14x14x64 -> 7x7x64
    h_pool2_flat = self.flatten_layer(h_pool2)  # 7x7x64(3136) -> 3136
    h_fc1 = self.fc_layer_1(h_pool2_flat)       # 3136 -> 1024
    logits = self.output_layer(h_fc1)           # 1024 -> 10
    y_pred = tf.nn.softmax(logits)

    return y_pred, logits


분류 문제 영역에서 손실함수로는 크로스 엔트로피를 많이 사용하는데, 크로스 엔터로피 손실함수 정의에서는 tf.nn라이브러리에서 내장하고 있는 softmax_cross_entropy_with_logits라는 함수를 사용해서 더 간단하게 코드를 구현했습니다.

이 때에 첫번째 인자값으로는 y-pred 예측값을 사용하면 안되고, 꼭 로지트값을 사용해야 하는데, 로지트값을 넣어주면 함수 내부에서 자동으로 소프트맥스를 적용한 후에 크로스엔트로피를 계산해줍니다.

 예전 실습할 때에, 이 함수를 사용하지 않고, 크로스 엔트로피 함수를 정의한 적이 있는데, 이보다 코드가 상당히 복잡해집니다. 두번째 인자인 참값 y는 one-hot-encoding된 참값 레이블입니다.


In [ ]:
# cross-entropy 손실 함수를 정의합니다.
@tf.function
def cross_entropy_loss(logits, y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))


옵티마이저로는 아담 옵티마이저를 선언했습니다. 이 때에 러닝레이트는 1e-4라는 표현을 사용했는데, 이것은 10^-4, 즉 0.0001이라는 의미입니다. 이렇게 숫자를 표현하는 방식을 scientific notation이라고 합니다.


In [ ]:
# 최적화를 위한 Adam 옵티마이저를 정의합니다.
optimizer = tf.optimizers.Adam(1e-4)



다음으로 최적화 학습을 진행할 train_step함수를 정의합니다. 많이 반복해서 나왔던 대로, loss손실함수 값을 trainable_variable들로 미분한 후에, 이 그래디언트 값들로 trainiable_variable들을 업데이트 해주는 것입니다.

In [ ]:
# 최적화를 위한 function을 정의합니다.
@tf.function
def train_step(model, x, y):
  with tf.GradientTape() as tape:
    y_pred, logits = model(x)
    loss = cross_entropy_loss(logits, y)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))


다음에는 학습이 끝나고 모델의 정확도를 측정해주는 함수입니다.

Tf.argmax는 최대값이 나오는 위치를 가르쳐주는 것인데, 참값 y는 one-hot-encoding되어 있으니까 본인에 해당되는 레이블이 있는 위치만 1이고 나머지는 0으로 되어 있는 벡터일 것이고, y_pred는 소프트맥스 결과니까 레이블의 확률을 나타내는 벡터일 것입니다. 그래서, argmax 최대값의 위치가 같다면 제대로 예측한 것이 되는 것입니다. Tf.equal은 두개의 텐서를 비교해서 같은 위치는 true, 틀린 위치는 false로 출력해주는 함수입니다.
이 값을 tf.cast함수로 float type으로 바꾼 후에, tf.reduce_mean 함수로 평균값을 구해주면 0부터 1 사이 값이 나오게 되고, 1에 가까울수록 정확도가 100%에 가깝게 됩니다.


In [ ]:
# 모델의 정확도를 출력하는 함수를 정의합니다.
@tf.function
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy


이제 CNN 모델 클래스의 인스턴스를 선언해주고, 실제 학습을 시작합니다.

Mini-batch training을 위해 50개씩 쪼개 놓은 60000개 데이터에 대해,순차적으로 10000번의 학습을 수행합니다.
10000번의 Mini-batch 학습을 수행하는 도중, 100번째마다 평균 손실함수값을 표시해줍니다.


In [ ]:
# Convolutional Neural Networks(CNN) 모델을 선언합니다.
CNN_model = CNN()

# 10000 Step만큼 최적화를 수행합니다.
for i in range(10000):
  # 50개씩 MNIST 데이터를 불러옵니다.
  batch_x, batch_y = next(train_data_iter)
  # 100 Step마다 training 데이터셋에 대한 정확도를 출력합니다.
  if i % 100 == 0:
    train_accuracy = compute_accuracy(CNN_model(batch_x)[0], batch_y)
    print("반복(Epoch): %d, 트레이닝 데이터 정확도: %f" % (i, train_accuracy))
  # 옵티마이저를 실행해 파라미터를 한스텝 업데이트합니다.
  train_step(CNN_model, batch_x, batch_y)


모든 학습이 끝나면 테스트 데이터로 학습의 정확도를 계산해서 출력합니다.


In [ ]:
# 학습이 끝나면 학습된 모델의 정확도를 출력합니다.
# 약 99% 정확도가 출력됨.
print("정확도(Accuracy): %f" % compute_accuracy(CNN_model(x_test)[0], y_test))

# 과제: tensorboard 시각화의 추가
예제코드 4.4에서 배운 tensorboard 시각화 기능을 위의 코드에 추가해보기



1.   create_file_writer함수로 train 및 test시의 텐서보드 summary 정보들을 저장할 폴더 경로를 설정하고 FileWriter를 선언(train_summary_writer와 test_sumary_writer).
2.   train_step 함수 정의에 매 step마다 텐서보드 로그에 tf.summary.scalar로 'loss'값을 기록하고, tf.summary.image로 'training image'를 기록하는 코드 추가.
3. 학습이 끝난후 모델 정확도를 계산하는 compute_accuracy 함수 정의에 'accuracy'를 기록하는 코드 추가.
4. 10000번 step의 최적화를 실행할 때에 train_summary_writer에 과정이 기록되도록 함.
5. 학습이 끝난후 compute_accuracy 계산시에 test_summary_writer에 accuracy가 기록되도록 함.
6. '%load_ext tensorboard' 명령어를 통해 Colab tensorboard extension을 불러옴.
7. '%tensorboard --logdir ./tensorboard_log' 명령어를 통해 학습과정을 시각화 해보기